### Import Libraries

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import os
import shutil

### Load and Normalize the Dataset

In [7]:
# Load the dataset
file_path = 'D:\\Github Mikezxc\\Big-data-stock-real-time-platform\\merged_data_with_ma.csv'
merged_df = pd.read_csv(file_path)

# Normalize data
price_scaler = MinMaxScaler()
merged_df[['close']] = price_scaler.fit_transform(merged_df[['close']])

feature_scaler = MinMaxScaler()
merged_df[['MA30', 'MA90']] = feature_scaler.fit_transform(merged_df[['MA30', 'MA90']])


### Prepare Sequences for LSTM

In [8]:
# Prepare sequences for LSTM
def create_sequences(df, time_steps=30):
    sequences = []
    labels = []
    for i in range(len(df) - time_steps):
        sequence = df[['close', 'MA30', 'MA90']].iloc[i:i+time_steps].values
        label = df['close'].iloc[i+time_steps]
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)


### Define the LSTM HyperModel

In [9]:
# Define the LSTM HyperModel
class LSTMHyperModel(kt.HyperModel):
    def build(self, hp):
        model = Sequential()
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(LSTM(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), 
                           return_sequences=(i != hp.Int('num_layers', 1, 3) - 1), input_shape=(time_steps, 3)))
        model.add(Dense(1))  # Output layer should match the number of features
        model.compile(optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)),
            loss='mean_squared_error')
        return model


### Train and Evaluate the Model for Each Ticker

In [10]:
# Train and evaluate the model for each ticker
tickers = merged_df['ticker'].unique()

for ticker in tickers:
    ticker_df = merged_df[merged_df['ticker'] == ticker].dropna()
    
    # Create sequences and labels
    time_steps = 30
    X, y = create_sequences(ticker_df, time_steps)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Initialize RandomSearch Tuner
    tuner = kt.RandomSearch(
        LSTMHyperModel(),
        objective='val_loss',
        max_trials=20,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'lstm_stock_model_{ticker}'
    )

    # Perform hyperparameter search
    tuner.search(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

    # Retrieve the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"Ticker: {ticker}")
    print(f"The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')}.")
    for i in range(best_hps.get('num_layers')):
        print(f"Layer {i + 1}: {best_hps.get(f'units_{i}')} units")
    print(f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.")

    # Build the model with the optimal hyperparameters
    model = tuner.hypermodel.build(best_hps)

    # Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

    # Create directory for ticker if it doesn't exist
    os.makedirs(f'models/{ticker}', exist_ok=True)

    # Save the model
    model.save(f'models/{ticker}/lstm_stock_model_best_{ticker}.h5')

    # Save the scalers
    joblib.dump(price_scaler, f'models/{ticker}/{ticker}_price_scaler.pkl')
    joblib.dump(feature_scaler, f'models/{ticker}/{ticker}_feature_scaler.pkl')

    # Save the history for plotting
    with open(f'models/{ticker}/history_{ticker}.pkl', 'wb') as file:
        joblib.dump(history.history, file)

    # Make predictions
    y_pred = model.predict(X_test)

    # Inverse transform the predictions and the actual values
    y_pred = price_scaler.inverse_transform(y_pred)
    y_test = price_scaler.inverse_transform(y_test.reshape(-1, 1))

    # Save the predictions for plotting
    np.save(f'models/{ticker}/y_test_{ticker}.npy', y_test)
    np.save(f'models/{ticker}/y_pred_{ticker}.npy', y_pred)


Reloading Tuner from my_dir\lstm_stock_model_AAPL\tuner0.json
Ticker: AAPL
The hyperparameter search is complete. The optimal number of layers is 1.
Layer 1: 288 units
The optimal learning rate for the optimizer is 0.0020813994011954795.
Epoch 1/50


c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0013 - val_loss: 3.3460e-04
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 7.5810e-05 - val_loss: 2.0958e-04
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 6.5130e-05 - val_loss: 1.3651e-04
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 4.2135e-05 - val_loss: 1.0275e-04
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 4.0367e-05 - val_loss: 2.0218e-04
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 5.8996e-05 - val_loss: 9.7610e-05
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 4.1843e-05 - val_loss: 8.9712e-05
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 2.8758e-05 - val_loss: 3.7544e-04
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 4.1502e-05 - val_loss: 7.8145e-05
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 2.9427e-05 - val_loss: 1.5804e-04
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 3.5406e-05 - val_loss: 1

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Reloading Tuner from my_dir\lstm_stock_model_GOOGL\tuner0.json
Ticker: GOOGL
The hyperparameter search is complete. The optimal number of layers is 3.
Layer 1: 32 units
Layer 2: 32 units
Layer 3: 64 units
The optimal learning rate for the optimizer is 0.0035065850163276276.
Epoch 1/50


c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - loss: 0.0152 - val_loss: 5.4507e-04
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - loss: 3.0260e-04 - val_loss: 3.8514e-04
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 1.6479e-04 - val_loss: 1.7283e-04
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 1.6813e-04 - val_loss: 5.4747e-04
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 2.9304e-04 - val_loss: 2.4356e-04
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 1.4418e-04 - val_loss: 1.3281e-04
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - loss: 8.7106e-05 - val_loss: 3.0747e-04
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - loss: 2.2336e-04 - val_loss: 4.0829e-04
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 1.4092e-04 - val_loss: 1.0938e-04
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 1.1906e-04 - val_loss: 1.2764e-04
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 8.6907e-05 - 

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
Reloading Tuner from my_dir\lstm_stock_model_TSLA\tuner0.json
Ticker: TSLA
The hyperparameter search is complete. The optimal number of layers is 3.
Layer 1: 160 units
Layer 2: 352 units
Layer 3: 160 units
The optimal learning rate for the optimizer is 0.00034786388011680107.
Epoch 1/50


c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 12s 140ms/step - loss: 0.0131 - val_loss: 0.0059
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 136ms/step - loss: 0.0015 - val_loss: 0.0031
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 135ms/step - loss: 0.0014 - val_loss: 0.0020
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 123ms/step - loss: 0.0010 - val_loss: 0.0017
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 123ms/step - loss: 8.6364e-04 - val_loss: 0.0015
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 121ms/step - loss: 0.0011 - val_loss: 0.0031
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 122ms/step - loss: 7.6213e-04 - val_loss: 0.0025
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 120ms/step - loss: 6.0977e-04 - val_loss: 0.0017
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 120ms/step - loss: 6.0767e-04 - val_loss: 9.4361e-04
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 117ms/step - loss: 5.5543e-04 - val_loss: 9.3140e-04
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 117ms/step - loss: 6.2171e-04 - val_loss: 8.4340e-04
Epoch 12/50
60/60 ━━━━━━━━

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step


### Plotting the Results

In [15]:
# Plotting the training and validation loss
def plot_training_validation_loss(ticker):
    with open(f'models/{ticker}/history_{ticker}.pkl', 'rb') as file:
        history = joblib.load(file)

    plt.figure(figsize=(14, 5))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {ticker}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'models/{ticker}/training_validation_loss_{ticker}.png')  # Save the plot
    plt.close()

# Plotting the stock price prediction
def plot_stock_price_prediction(ticker):
    y_test = np.load(f'models/{ticker}/y_test_{ticker}.npy')
    y_pred = np.load(f'models/{ticker}/y_pred_{ticker}.npy')

    plt.figure(figsize=(14, 5))
    plt.plot(y_test, color='blue', label='Actual Stock Price')
    plt.plot(y_pred, color='red', label='Predicted Stock Price')
    plt.title(f'Stock Price Prediction for {ticker}')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.savefig(f'models/{ticker}/stock_price_prediction_{ticker}.png')  # Save the plot
    plt.close()

# Plot results for each ticker
for ticker in tickers:
    plot_training_validation_loss(ticker)
    plot_stock_price_prediction(ticker)


# Load the dataset
file_path = 'D:\\Github Mikezxc\\Big-data-stock-real-time-platform\\merged_data_with_ma.csv'
merged_df = pd.read_csv(file_path)

# Normalize data
price_scaler = MinMaxScaler()
merged_df[['close']] = price_scaler.fit_transform(merged_df[['close']])

feature_scaler = MinMaxScaler()
merged_df[['MA30', 'MA90']] = feature_scaler.fit_transform(merged_df[['MA30', 'MA90']])

# Prepare sequences for LSTM
def create_sequences(df, time_steps=30):
    sequences = []
    labels = []
    for i in range(len(df) - time_steps):
        sequence = df[['close', 'MA30', 'MA90']].iloc[i:i+time_steps].values
        label = df['close'].iloc[i+time_steps]
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Define the LSTM HyperModel
class LSTMHyperModel(kt.HyperModel):
    def build(self, hp):
        model = Sequential()
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(LSTM(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), 
                           return_sequences=(i != hp.Int('num_layers', 1, 3) - 1), input_shape=(time_steps, 3)))
        model.add(Dense(1))  # Output layer should match the number of features
        model.compile(optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)),
            loss='mean_squared_error')
        return model

# Train and evaluate the model for each ticker
tickers = merged_df['ticker'].unique()

for ticker in tickers:
    ticker_df = merged_df[merged_df['ticker'] == ticker].dropna()
    
    # Create sequences and labels
    time_steps = 30
    X, y = create_sequences(ticker_df, time_steps)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Initialize RandomSearch Tuner
    tuner = kt.RandomSearch(
        LSTMHyperModel(),
        objective='val_loss',
        max_trials=20,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'lstm_stock_model_{ticker}'
    )

    # Perform hyperparameter search
    tuner.search(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

    # Retrieve the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"Ticker: {ticker}")
    print(f"The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')}.")
    for i in range(best_hps.get('num_layers')):
        print(f"Layer {i + 1}: {best_hps.get(f'units_{i}')} units")
    print(f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.")

    # Build the model with the optimal hyperparameters
    model = tuner.hypermodel.build(best_hps)

    # Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

    # Save the model
    model.save(f'/mnt/data/lstm_stock_model_best_{ticker}.h5')

    # Save the scalers
    joblib.dump(price_scaler, f'/mnt/data/{ticker}_price_scaler.pkl')
    joblib.dump(feature_scaler, f'/mnt/data/{ticker}_feature_scaler.pkl')

    # Plot the training and validation loss
    plt.figure(figsize=(14, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {ticker}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Make predictions
    y_pred = model.predict(X_test)

    # Inverse transform the predictions and the actual values
    y_pred = price_scaler.inverse_transform(y_pred)
    y_test = price_scaler.inverse_transform(y_test.reshape(-1, 1))

    # Plot the results
    plt.figure(figsize=(14, 5))
    plt.plot(y_test, color='blue', label='Actual Stock Price')
    plt.plot(y_pred, color='red', label='Predicted Stock Price')
    plt.title(f'Stock Price Prediction for {ticker}')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()